We want to:
- Ask the user for a logical expression (e.g., A and B or not A or B)

- Figure out what the variables are (A, B, etc.)

- Generate all combinations of True/False for those variables

- Evaluate the expression for each combination

- Print the truth table

\

## Pipeline:
* We need to generate all combinations of T (1) and F (0) for our variables A and B. So, we use "itertools.product"

In [7]:
from itertools import product

## Step 1: Extract variable names:

  - Split the expression into words
  - Replace special symbols with spaces (to separate tokens)
  - Keep and return a sorted list of variables

In [8]:
def extract_variables(expr):
    keywords = {"and", "or", "not", "True", "False"}
    symbols_to_space = ["(", ")", "⇒", "→", "↔", "<=>"]
    for sym in symbols_to_space:
        expr = expr.replace(sym, " ")
    tokens = expr.split()
    variables = []
    for token in tokens:
        if token.isalpha() and token not in keywords and token not in variables:
            variables.append(token)
    return sorted(variables)

## Step 2: Convert symbols like "→", "<=>" into Python logic:
  - Replace symbols like ¬, ∧, ∨, →, and ↔ to placeholders like  not, and, =>, <=>
  - Replace => with **`not A or B`**, and <=> with **`(A and B) or (not A and not B)`**
    - Scan for "=>" and rewrite it as: "not A or B"
    - Scan for "<=>" and rewrite it as: "(A and B) or (not A and not B)"


In [9]:
def preprocess(expr):
    expr = expr.replace("¬", "not ")
    expr = expr.replace("∧", "and")
    expr = expr.replace("∨", "or")
    expr = expr.replace("→", "=>").replace("⇒", "=>")
    expr = expr.replace("↔", "<=>")

    # Handle implication A => B → (not A or B)
    expr = expr.replace("<=>", " <=> ")  # spacing to split easily
    expr = expr.replace("=>", " => ")

    tokens = expr.split()
    while "=>" in tokens:
        idx = tokens.index("=>")
        left = tokens[idx - 1]
        right = tokens[idx + 1]
        tokens[idx - 1:idx + 2] = [f"(not {left} or {right})"]
    while "<=>" in tokens:
        idx = tokens.index("<=>")
        left = tokens[idx - 1]
        right = tokens[idx + 1]
        tokens[idx - 1:idx + 2] = [f"(({left} and {right}) or (not {left} and not {right}))"]
    return " ".join(tokens)

## Step 3: Replace variables with values and evaluate the expression:
  - Take an expression like "not A or B"
  - Replace A and B with their values (T or F)
  - Use eval() to calculate the result

  Example:
  - For "not A or B", if A = False, B = True, expression becomes "not False or True". So, eval() will produce the result = True

In [10]:
def evaluate(expr, values):
    for var in values:
        expr = expr.replace(var, str(values[var]))
    try:
        return eval(expr)
    except:
        return "Error"

## Step 4: Put everything together to print the truth table:
  - Use the above functions to convert inputs into Python-evaluable format, and to find all variable names
  - Generate all combinations of True(1)/False(0) for those variables
  - Evaluate expression for each case
  - Print a formatted truth table (for visual output)

In [11]:
def generate_truth_table(expr):
    original_expr = expr
    expr = preprocess(expr)
    variables = extract_variables(original_expr)

    print("Variables found:", variables)
    print("\nTruth Table:")
    print(" | ".join(variables) + " | Result")
    print("-" * (6 * len(variables) + 10))

    for combo in product([False, True], repeat=len(variables)):
        values = dict(zip(variables, combo))
        result = evaluate(expr, values)
        row = " | ".join(str(values[v]) for v in variables)
        print(f"{row} | {result}")

## Step 5: Ask the user to input a logical expression:
  - Give instructions to take input
  - Call the above function to generate the table

In [14]:
print("Enter a logical expression using: and, or, not, parentheses")
print("Example: (A and B) or not C\n")

user_expr = input("Your expression: ")

generate_truth_table(user_expr)


Enter a logical expression using: and, or, not, parentheses
Example: (A and B) or not C

Your expression: A and B
Variables found: ['A', 'B']

Truth Table:
A | B | Result
----------------------
False | False | False
False | True | False
True | False | False
True | True | True
